# Tutorial on Wannier Berri - basic

In this tutorial we will see how to compute bands, Berry curvature and anomalous Hall conductivity  using WannierBerri code

\begin{equation}
\sigma^{\rm AHC}_{xy} = -\frac{e^2}{\hbar} \sum_n^{\rm occ} \int\frac{d\mathbf{k}}{(2\pi)^3} \Omega^n_\gamma
\label{eq:ahc}\tag{1}
\end{equation}

Equation (\ref{eq:ahc}) defines AHC as a Fermi-sea integral of Berry curvature

In [ ]:
# Preliminary (Do only once)

# Set environment variables - not mandatory but recommended
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1' 
os.environ['MKL_NUM_THREADS'] = '1'


import wannierberri as wberri
import numpy as np
import scipy
import matplotlib.pyplot as plt

#  This block is needed if you run this cell for a second time
#  because one cannot initiate two parallel environments at a time
try:
    parallel.shutdown()   
except NameError:
    pass

parallel = wberri.Parallel(num_cpus=2)


In [ ]:
# Importing data from wannier90
system=wberri.System_w90(
                        seedname='w90_files/Fe',
                        berry=True,   # needed to calculate "external terms" of Berry connection or curvature , reads ".mmn" file
                        spin = True , # needed for spin properties, reads ".spn" file
                        use_wcc_phase=True  # 
                        )

generators=["Inversion","C4z","TimeReversal*C2x"]
system.set_symmetry(generators)


In [ ]:
# Evaluate bands, Berry curvature and spin along a path GHPNG


path=wberri.Path(system,
                 k_nodes=[
        [0.0000, 0.0000, 0.0000 ],   #  G 
        [0.500 ,-0.5000, -0.5000],   #  H 
        [0.7500, 0.2500, -0.2500],   #  P 
        [0.5000, 0.0000, -0.5000],   #  N 
        [0.0000, 0.0000, 0.000  ] ] , #  G 
                 labels=["G","H","P","N","G"],
                 length=200 )   # length [ Ang] ~= 2*pi/dk


path_result=wberri.tabulate(system,
                            grid=path,
                            quantities = ["spin","berry"],
                            parallel = parallel)


In [ ]:
# plot the bands and compare with QuantumEspresso
EF = 12.6

# Import the pre-computed bands from quantum espresso
A = np.loadtxt(open("bands/Fe_bands_pw.dat","r"))
bohr_ang = scipy.constants.physical_constants['Bohr radius'][0] / 1e-10
alatt = 5.4235* bohr_ang
A[:,0]*= 2*np.pi/alatt
A[:,1]-=EF
# plot it as dots
plt.scatter (A[:,0],A[:,1],s=5,label = "QE")


path_result.plot_path_fat( path,
              quantity=None,
              save_file="Fe_bands+QE.pdf",
              Eshift=EF,
              Emin=-10,  Emax=50,
              iband=None,
              mode="fatband",
              fatfactor=20,
              cut_k=False, 
              close_fig=False,
              show_fig=False,
              label = "WB"
              )


plt.legend()
plt.show()
plt.close()


In [ ]:
# plot the bands and compare with wannier90
A = np.loadtxt(open("bands/Fe_bands_w90.dat","r"))
plt.scatter (A[:,0],A[:,1],s=5,label = "W90")

path_result.plot_path_fat( path,
              quantity=None,
              save_file="Fe_bands+w90.pdf",
              Eshift=0,
              Emin=4,  Emax=20,
              iband=None,
              mode="fatband",
              fatfactor=20,
              cut_k=False, 
              close_fig=False,
              show_fig=False,
              label = "WB"
              )

plt.legend()

plt.show()
plt.close()

In [ ]:
# Plot bands and their spin (z component)

path_result.plot_path_fat( path,
              quantity='spin',
              component='z',
              save_file="Fe_bands+spin.pdf",
              Eshift=0,
              Emin=4,  Emax=20,
              iband=None,
              mode="fatband",
              fatfactor=20,
              cut_k=False, 
              close_fig=True,
              show_fig=True
              )

In [ ]:
# plot the Berry curvature
path_result.plot_path_fat( path,
              quantity='berry',
              component='z',
              save_file="Fe_bands+berry.pdf",
              Eshift=0,
              Emin=4,  Emax=20,
              iband=None,
              mode="fatband",
              fatfactor=4,
              cut_k=False, 
              close_fig=True,
              show_fig=True
              )

# The size of the dots are the magnitude of BC on a logarithmic scale

In [ ]:
# Set a grid 
grid = wberri.Grid(system, length=100 )   # length [ Ang] ~= 2*pi/dk
#grid = wberri.Grid(system, NK=[24,24,24], NKFFT=4)
#grid = wberri.Grid(system, NKdiv=6, NKFFT=4)

In [ ]:
tab_result=wberri.tabulate(system,
                             grid=grid,
                             quantities = ["spin","berry"],
                             ibands=np.arange(0,18),
                             parallel = parallel)

In [ ]:
tab_result.fermiSurfer(quantity= "berry",component="z",efermi=EF,frmsf_name = "Fe_berry_z.frmsf")
!ls


In [ ]:
!fermisurfer Fe_berry_z.frmsf

In [ ]:
Energy = tab_result.get_data(iband=6, quantity='Energy')
berry  = tab_result.get_data(iband=4, quantity='berry',component='z')
print(berry.shape,Energy.shape)

In [ ]:
# example : find the total Berry curvature of occupied states
# on the plane (k1,k2), k3=const (in reduced coordinates
Berry_occ = 0
k3 = 9
for ib in range(18):
    Energy = tab_result.get_data(iband=ib, quantity='Energy') [:,:,k3]
    berry  = tab_result.get_data(iband=ib, quantity='berry',component='z') [:,:,k3]
    berry [Energy>EF] = 0
    Berry_occ += berry
    plt.contour(Energy,[EF],linewidths=0.5,colors='black')
shw = plt.imshow(Berry_occ,vmin=-10,vmax=10,cmap="jet")
bar = plt.colorbar(shw)

In [ ]:
# Integrate the Berry curvature to get AHC
Efermi = np.linspace(12,13,1001)
wberri.integrate(system,
            grid=grid,
            Efermi=Efermi, 
            smearEf=300,  # Kelvin
            quantities=["ahc"],
            parallel=parallel,
            use_irred_kpt=True, symmetrize=True,
            adpt_num_iter=10,
            fout_name='Fe',
            restart=False,
            )

In [ ]:
!ls

In [ ]:
#plot results from different iterations
for i in range(10):
    a = np.loadtxt(f"Fe-ahc_iter-{i:04d}.dat")
    plt.plot(a[:,0],a[:,3],label = f"iteration-{i}")
#plt.ylim(-1000,1000)
plt.legend()
plt.show()

In [ ]:
# Do some more iterations, using 'K_list.pickle' file
wberri.integrate(system,
            grid=grid,
            Efermi=Efermi, 
            smearEf=300,  # Kelvin
            quantities=["ahc"],
            parallel=parallel,
            use_irred_kpt=True, symmetrize=True,
            adpt_num_iter=10,
            fout_name='Fe',
            restart=True,
            )

In [ ]:
#plot results from new iterations
for i in range(10,20):
    a = np.loadtxt(f"Fe-ahc_iter-{i:04d}.dat")
    plt.plot(a[:,0],a[:,3],label = f"iteration-{i}")
#plt.ylim(-1000,1000)
plt.legend()
plt.show()